# MMIS 692: Production Planning

## Formulate LP model

#### Parameters of the LP model:
- $D_i$ is the demand (number of units required) for product $i$.
- $C_i$ is the cost (in dollars) for producing each unit of product $i$ *in-house*.
- $P_i$ is the price (in dollars) for purchasing each unit of product $i$.
- $m_i$ is the machining time (in minutes) required to produce each unit of product $i$.
- $a_i$ is the assembly time (in minutes) required to produce each unit of product $i$.
- $f_i$ is the finishing time (in minutes) required to produce each unit of product $i$.
- $T_m$ is the machining time available (in minutes)
- $T_a$ is the assembly time available (in minutes)
- $T_f$ is the finishing time available (in minutes)

#### Decision Variables (all variables non-negative):
- $X_i$ : for $i=1,2,...,5$: number of units of product $i$ produced.
- $Y_i$ : for $i=1,2,...,5$: number of units of product $i$ purchased.

#### Objective function:
- Minimize Cost = $\sum_{i=1}^5 \  (C_i X_i  +  P_i Y_i)$

#### Constraints:
- Demand: $X_i + Y_i \ge D_i$ for $i=1,2,..,5$
- Machining time: $\sum_{i=1}^5 \  m_i X_i  \le  T_m$
- Assembly time: $\sum_{i=1}^5 \  a_i X_i  \le  T_a$
- Finishing time: $\sum_{i=1}^5 \  f_i X_i  \le  T_f$



## Import libraries

We shall use the Python library *PuLP* (https://pypi.org/project/PuLP/) for creating and solving the LP model. 

In [1]:
# ! pip install pulp # install if necessary
from pulp import * # for LP model
import pandas as pd # for data handle

## Get data

We shall extract *CSV* files from the *zip* file with data and read the data we need for this task into *pandas* dataframes:
- ***product***: with data from '*production_planning.product*'.csv'
- ***resource***: with data from '*production_planning.resource.csv*'

In [2]:
# ! unzip '/content/drive/MyDrive/Colab Notebooks/courses/MMIS692_Summer2021/data.MMIS692.Sample.zip'
product = pd.read_csv('production_planning.product.csv') # dataframe with product data
resource = pd.read_csv('production_planning.resource.csv') # dataframe with resource data
# ! rm *.csv # remove all data files to avoid clutter

In [3]:
product # display product data

,parameter,P1,P2,P3,P4,P5
0,demand,7000,9000,8000,6000,4000
1,cost_inhouse,49,84,50,97,70
2,cost_outsource,68,118,80,124,100
3,machine_time,1,2,2,3,4
4,assembly_time,3,4,2,2,3
5,finishing_time,2,4,4,2,1


In [4]:
resource # display resource data

,resource,available_hours,hourly_cost
0,machine_time,1000,300
1,assembly_time,1293,120
2,finishing_time,1306,240


## Get LP parameters
We shall read in parameter values for our LP model from  dataframes.

In [5]:
PRODUCTS = list(product)[1:] # list of products (all but first column header)
N = len(PRODUCTS) # number of products
D, C, P, m, a, f = product.values[:,1:].tolist() # parameter values

RESOURCES = resource.resource.tolist() # list of resources
Tm, Ta, Tf = (60*resource.available_hours).tolist() # parameter values
HOURLY_COST = resource.hourly_cost.tolist()

## Create LP model

In [6]:
LP_file = 'MMIS692_prodution_planning.lp' # name of LP model file
prob = LpProblem(LP_file, LpMinimize) # Create LP model object

### Define decision variables

In [7]:
X = [LpVariable(f'x_{i+1}',0) for i in range(N)] # quantities produced
Y = [LpVariable(f'y_{i+1}',0) for i in range(N)] # quantities purchased

### Specify objective function

In [8]:
prob += lpSum(C[i]*X[i] + P[i]*Y[i] for i in range(N)) # objective function

### Specify demand constraints

In [9]:
for i in range(N):
    prob += X[i] + Y[i] >= D[i] , 'Demand for '+ PRODUCTS[i] # demand for product i

### Add resource availability constraints

In [10]:
for r, t, q in zip(RESOURCES, [m, a, f], [Tm, Ta, Tf]): # for each resource
    prob += sum(t[i]*X[i] for i in range(N)) <= q, r +'_availability' # add resource availability constraint

### Save LP model

In [11]:
prob.writeLP(LP_file) # write model to LP file
print(open(LP_file).read()) # show LP model

\* MMIS692_prodution_planning.lp *\
Minimize
OBJ: 49 x_1 + 84 x_2 + 50 x_3 + 97 x_4 + 70 x_5 + 68 y_1 + 118 y_2 + 80 y_3
 + 124 y_4 + 100 y_5
Subject To
Demand_for_P1: x_1 + y_1 >= 7000
Demand_for_P2: x_2 + y_2 >= 9000
Demand_for_P3: x_3 + y_3 >= 8000
Demand_for_P4: x_4 + y_4 >= 6000
Demand_for_P5: x_5 + y_5 >= 4000
assembly_time_availability: 3 x_1 + 4 x_2 + 2 x_3 + 2 x_4 + 3 x_5 <= 77580
finishing_time_availability: 2 x_1 + 4 x_2 + 4 x_3 + 2 x_4 + x_5 <= 78360
machine_time_availability: x_1 + 2 x_2 + 2 x_3 + 3 x_4 + 4 x_5 <= 60000
End



## Solve LP model

In [12]:
prob.solve() # solve problem
status = LpStatus[prob.status] # optimal found?
print("Solution: ", status)

min_cost = value(prob.objective) # objective function value
print(f'Minimum cost = $ {min_cost:,.2f}') # show optimal cost

Solution:  Optimal
Minimum cost = $ 2,553,400.00


### Optimal decision variable values

In [13]:
# get optimal values for decision variables
X_opt = [x.varValue for x in X] # optimal quantities produced
Y_opt = [y.varValue for y in Y] # optimal quantities purchased

# Save and display optimal quantities in a dataframe
opt_qty = pd.DataFrame([X_opt, Y_opt], columns=PRODUCTS).round(2) # convert to dataframe
opt_qty.insert(0, 'Quantity', ['Produced', 'Purchased']) # add first column
opt_qty.to_csv("optimal_quantities.csv", index=False) # save results
print('Optimal quantities:')
opt_qty # display optimal quantities

Optimal quantities:


,Quantity,P1,P2,P3,P4,P5
0,Produced,4050.0,9000.0,4935.0,6000.0,2520.0
1,Purchased,2950.0,0.0,3065.0,0.0,1480.0


### Resources used 

In [14]:
resource_use = [] # list with results
for r in RESOURCES: # for each resource
    name = f'{r}_availability' # availability constraint name
    c = prob.constraints[name] # details for constraint
    available = -c.constant # available minutes (RHS of the constraint)
    unused = c.slack # minutes not used
    used = available - unused # minutes used
    resource_use.append([r, used, available, unused]) # append results for resource
# convert results to dataframe
resource_use = pd.DataFrame(resource_use, columns = ['resource', 'used', 'available', 'unused'])
resource_use.to_csv('resource_use.csv', index=False) # sane results
resource_use # show results

,resource,used,available,unused
0,machine_time,60000.0,60000,-0.0
1,assembly_time,77580.0,77580,-0.0
2,finishing_time,78360.0,78360,-0.0


## Sensitivity analysis

In [15]:
max_price = [] # results with maximum cost per additional hour
for r, hc in zip(RESOURCES, HOURLY_COST): # for each resource and its hourly cost
    name = f'{r}_availability' # constraint name
    c = prob.constraints[name] # details for constraint
    shadow_price = -c.pi # cost savings per minute of resource
    savings = 60*shadow_price # cost savings per additional hour of resource
    price = hc + savings # maximum price for an additional hour of resource
    max_price.append([r, hc, savings, price])
# convert results to dataframe
max_price = pd.DataFrame(max_price, columns = ['resource', 'cost per hour', 'savings per hour', 'max price'])
max_price.to_csv('max_price.csv', index=False) # save results
max_price # show results

,resource,cost per hour,savings per hour,max price
0,machine_time,300,300.0,600.0
1,assembly_time,120,120.0,240.0
2,finishing_time,240,240.0,480.0
